# University Certificate in Artificial Intelligence (Hands on AI, Third Challenge, 2022-2023, UMONS)
# Forecasting methods



In [ ]:
if False:
    # The dataset and package "main" are in the GitHub repository
    !git clone https://github.com/bsouhaib/Hands-On-AI-2022-Challenge3.git
    %cd Hands-On-AI-2022-Challenge3/Exercises
    # These packages are not installed by default
    !pip3 install pmdarima supersmoother

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = [10, 5]
sns.set_theme()

## Simulated time series (ARMA)

We will simulate a time series from the following ARMA process:
$$
y_t = 0.75 y_{t-1} + 0.25 y_{t-2} + 0.65 \varepsilon_{t-1} + 0.35 \varepsilon_{t-2} + \varepsilon_t
$$

In [ ]:
import statsmodels.api as sm

np.random.seed(12345)

T = 200

# ARMA parameters
arparams = np.array([0.75, -0.25])
maparams = np.array([0.65, 0.35])
ar_term = np.r_[1, -arparams]  # add zero-lag and negate
ma_term = np.r_[1, maparams]  # add zero-lag

arma_process = sm.tsa.ArmaProcess(ar_term, ma_term)

y = arma_process.generate_sample(T)
plt.plot(y)

* Plot the autocorrelation function (ACF) of the simulated series for the first 10 lags.


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

# Hint: use plot_acf

###
plot_acf(y, lags=10)

* Fit an ARIMA model to the simulated time series using auto_arima. Do you recover the true parameters? To obtain details about your model fit, you can use the following functions: summary(), arparams(), and maparams().

In [ ]:
from pmdarima.arima import auto_arima

###
model = auto_arima(y)

In [ ]:
def result_table(model):
    return pd.DataFrame({
        'Fitted parameters': [str(model.arparams()), str(model.maparams())],
        'True parameters': [-ar_term[1:], ma_term[1:]],
    }, index=['AR', 'MA'])

result_table(model)

In [ ]:
model.summary()

* Increase the size of the simulated series (e.g. $T=1000$) and refit an ARIMA model using auto_arima. Do you recover the true parameters?

In [ ]:
###
T = 1000
y = arma_process.generate_sample(T)
model = auto_arima(y, information_criterion="bic")

result_table(model)

* Change the default parameters of the auto_arima function to get faster results. Eploit the fact that you know the "true" data generating process.

In [ ]:
# Point d'interrogation partout
model = auto_arima(
    y,
    d=0,
    start_p=2,
    max_p=2,
    start_q=2,
    max_q=2,
    D=0,
    start_P=0,
    max_P=0,
    start_Q=0,
    max_Q=0,
    seasonal=False,
    information_criterion="bic",
)

result_table(model)

In [ ]:
model.summary()

## Real-world time series

In [ ]:
# Read the data file
DF = pd.read_csv("../data/public/train.csv", parse_dates=True)
DF["Day"] = pd.to_datetime(DF["Day"], format="%Y-%m-%d")
DF.set_index("Day", inplace=True)
DF = DF.asfreq("D")
DF.fillna(method="backfill", inplace=True)

DF

In [ ]:
# Select the series to consider
id_series_all = ["s001", "s002", "s003"]
DF_all = DF[id_series_all].copy()

DF_all

In [ ]:
HORIZON = 7 * 2
DF_train = DF_all[:-HORIZON]
DF_test = DF_all[-HORIZON:]

* Use auto.arima to compute forecasts for all series

In [ ]:
from pmdarima.arima import auto_arima

fcts_arima_list = list()

for id_series in id_series_all:
    print("======", id_series, "======")
    y = DF_train[id_series]
    model = auto_arima(
        y,
        d=0,
        start_p=0,
        max_p=1,
        start_q=0,
        max_q=1,
        D=0,
        start_P=0,
        max_P=2,
        start_Q=0,
        max_Q=0,
        m=7,
        trace=True,
    )
    f_arima = model.predict(HORIZON)
    f_arima.name = id_series
    fcts_arima_list.append(f_arima)

fcts_arima = pd.concat(fcts_arima_list, axis=1)

* Compute naive forecasts for all series

In [ ]:
fcts_mean_list = list()
fcts_naive_list = list()
fcts_snaive_list = list()

for series in id_series_all:
    series_train = DF_train[series]
    series_test = DF_test[series]

    T = len(series_train)

    ## Mean
    meanf = series_train.mean()
    f_mean = pd.Series([meanf for h in range(0, HORIZON)], index=series_test.index)
    f_mean.name = series

    fcts_mean_list.append(f_mean)

    ## Naive
    f_naive = series_train[-1]
    f_naive = pd.Series([f_naive for h in range(0, HORIZON)], index=series_test.index)
    f_naive.name = series

    fcts_naive_list.append(f_naive)

    ## Seasonal naive
    period = 7
    f_snaive = [series_train[T + h - period * ((HORIZON -1)//period + 1)] for h in range(0, HORIZON) ]
    f_snaive = pd.Series(f_snaive, index=series_test.index)
    f_snaive.name = series

    fcts_snaive_list.append(f_snaive)

In [ ]:
fcts_mean = pd.concat(fcts_mean_list, axis=1)
fcts_naive = pd.concat(fcts_naive_list, axis=1)
fcts_snaive = pd.concat(fcts_snaive_list, axis=1)

* Compute the Symmetric Mean Absolute Percentage Error (SMAPE) for all series.

In [ ]:
def smape(y_true, y_pred):
    assert (y_pred >= 0).all().all()
    denominator = (y_true + y_pred) / 200.0
    SAPE = np.abs(y_true - y_pred) / denominator
    SAPE[denominator == 0] = 0.0
    return SAPE.mean().mean()


print(smape(DF_test, fcts_mean))
print(smape(DF_test, fcts_naive))
print(smape(DF_test, fcts_snaive))
print(smape(DF_test, fcts_arima))

In [ ]:
id_series = "s001"
fcts_naive[id_series].plot(label="naive")
fcts_snaive[id_series].plot(label="snaive")
DF_test[id_series].plot(label="true")
fcts_arima[id_series].plot(label="arima")
plt.legend()

# Neural network forecast

In the following, we will consider two neural network architectures for forecasting. Your task is to play with all the hyperparameters to obtain the best out-of-sample forecasts, i.e. on the test set.

Some important hyperparameters include: n_simul (size of the dataset), LAG (the number of lagged values), LATENT_DIM (the number of units in the layer), BATCH_SIZE (number of samples per mini-batch), EPOCHS (the number of epochs), the optimizer and the early stop strategy.

In [ ]:
from pmdarima.arima import auto_arima
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

from main.utils.utils_methods import embed_data, plot_learning_curves
from main.utils.utils import mse, mae, mape, smape

%load_ext autoreload
%autoreload 2

* We will simulate a time series from a nonlinear stochastic process:


In [ ]:
n_simul = 1000
n_burn = 100
n = n_simul + n_burn
noise = np.random.normal(size=n)

y = np.zeros(n)
y[0] = 0
y[1] = 0
for t in range(2, n):
    y[t] = (
        0.3 * y[t - 1]
        + 0.6 * y[t - 2]
        + (0.1 - 0.9 * y[t - 1] + 0.8 * y[t - 2]) * (1 / (1 + np.exp(-10 * y[t - 1])))
        + noise[t]
    )

data = pd.DataFrame(y[n_burn:], columns=["series"])
plt.plot(data)

Choose which loss function you want to experiment with. It is used later in the code to fit and evaluate a neural network model.

In [ ]:
# Loss function to be used to optimize the model parameters
loss_fct = "mse"  # 'mae'

# Accuracy measure to be used to evaluate test predictions.
accuracy_measure = mse  # mae # mape # smape

In [ ]:
# The forecast horizon
HORIZON = 3

# The number of lagged values.
LAG = 4

# Data split
n = len(data)
n_train = int(0.6 * n)
n_valid = int(0.2 * n)
n_learn = n_train + n_valid

train = data[:n_train]
valid = data[n_train:n_learn]
test = data[n_learn:]

# From time series to input-output data (also called time series embedding)
(
    train_inputs,
    valid_inputs,
    test_inputs,
    X_train,
    y_train,
    X_valid,
    y_valid,
    X_test,
    y_test,
) = embed_data(train, valid, test, HORIZON, LAG, freq=None, variable="series")

In [ ]:
display(X_train.head())
display(y_train.head())

# Multioutput MLP

In [ ]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense

# Simple MLP with 1 hidden layer
def mlp_multioutput(X_train, y_train, X_valid, y_valid, LATENT_DIM, BATCH_SIZE, EPOCHS, LAG, HORIZON, 
                    loss, optimizer, earlystop, best_val, verbose):
  
    model = Sequential()
    model.add(Dense(LATENT_DIM, activation="relu", input_shape=(LAG,)))
    model.add(Dense(HORIZON))
    model.compile(optimizer=optimizer, loss=loss)
    
    history = model.fit(
        X_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_valid, y_valid),
        callbacks=[earlystop, best_val],
        verbose=verbose
    )
    return model, history

In [ ]:
#########################
file_header = "model_" + "mlp_multioutput"
verbose = 0

optimizer_adam = keras.optimizers.Adam(learning_rate=0.01)
earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience=50)

LATENT_DIM = 5  # 50   # number of units in the RNN layer
BATCH_SIZE = 32  # number of samples per mini-batch
EPOCHS = 200  # maximum number of times the training algorithm will cycle through all samples
loss = loss_fct

best_val = ModelCheckpoint(
    "../work/" + file_header + "_{epoch:02d}.h5",
    save_best_only=True,
    mode="min",
    save_freq="epoch",
    monitor="val_loss",
)
#########################

model_mlp_multioutput, history_mlp_multioutput = mlp_multioutput(
    X_train,
    y_train,
    X_valid,
    y_valid,
    LATENT_DIM=LATENT_DIM,
    BATCH_SIZE=BATCH_SIZE,
    EPOCHS=EPOCHS,
    LAG=LAG,
    HORIZON=HORIZON,
    loss=loss,
    optimizer=optimizer_adam,
    earlystop=earlystop,
    best_val=best_val,
    verbose=verbose,
)

plot_learning_curves(history_mlp_multioutput)

best_epoch = np.argmin(np.array(history_mlp_multioutput.history["val_loss"])) + 1
print("Best epoch:", best_epoch)
filepath = "../work/" + file_header + "_{:02d}.h5"
model_mlp_multioutput.load_weights(filepath.format(best_epoch))

## Recursive MLP


In [ ]:
#########################
file_header = "model_" + "mlp_recursive"
verbose = 0

optimizer_adam = keras.optimizers.Adam(learning_rate=0.01)
earlystop = EarlyStopping(monitor="val_loss", min_delta=0, patience=50)

LATENT_DIM = 5  # number of units in the RNN layer
BATCH_SIZE = 32  # number of samples per mini-batch
EPOCHS = 200  # maximum number of times the training algorithm will cycle through all samples
loss = loss_fct

best_val = ModelCheckpoint(
    "../work/" + file_header + "_{epoch:02d}.h5",
    save_best_only=True,
    mode="min",
    save_freq="epoch",
    monitor="val_loss",
)
#########################

(
    _,
    _,
    _,
    X_train_onestep,
    y_train_onestep,
    X_valid_onestep,
    y_valid_onestep,
    _,
    _,
) = embed_data(train, valid, test, 1, LAG, freq=None, variable="series")

# The recursive MLP is just a multioutput MLP with 1 output.
# However, the predictions given by the multioutput MLP and recursive MLP 
# are not created in the same way.
model_mlp_recursive, history_mlp_recursive = mlp_multioutput(
    X_train_onestep,
    y_train_onestep,
    X_valid_onestep,
    y_valid_onestep,
    LATENT_DIM=LATENT_DIM,
    BATCH_SIZE=BATCH_SIZE,
    EPOCHS=EPOCHS,
    LAG=LAG,
    HORIZON=1,
    loss=loss,
    optimizer=optimizer_adam,
    earlystop=earlystop,
    best_val=best_val,
    verbose=verbose,
)
plot_learning_curves(history_mlp_recursive)

best_epoch = np.argmin(np.array(history_mlp_recursive.history["val_loss"])) + 1
print("Best epoch:", best_epoch)
filepath = "../work/" + file_header + "_{:02d}.h5"
model_mlp_recursive.load_weights(filepath.format(best_epoch))

# Naive forecasts

In [ ]:
# len(X_test.values[:, -1])
predictions_naive = np.tile(X_test.values[:, -1], (HORIZON, 1)).T
predictions_naive = pd.DataFrame(
    predictions_naive, columns=[f"t+{t}" for t in range(1, HORIZON + 1)]
)
predictions_naive

# MLP forecasts

In [ ]:
predictions_mlp_multioutput = model_mlp_multioutput.predict(X_test)
predictions_mlp_multioutput = pd.DataFrame(
    predictions_mlp_multioutput, columns=[f"t+{t}" for t in range(1, HORIZON + 1)]
)

predictions_mlp_multioutput

In [ ]:
X_test

In [ ]:
for h in range(HORIZON):
    pred = model_mlp_recursive.predict(X_test)
    # `predictions_mlp_recursive` contains the predictions
    if h == 0:
        predictions_mlp_recursive = pred
    else:
        predictions_mlp_recursive = np.hstack((predictions_mlp_recursive, pred))
    # `X_test` is updated at each step
    X_test = pd.DataFrame(
        np.hstack((X_test.to_numpy()[:, 1:], pred)),
        index=X_test.index,
        columns=X_test.columns,
    )

predictions_mlp_recursive = pd.DataFrame(
    predictions_mlp_recursive, columns=[f"t+{t}" for t in range(1, HORIZON + 1)]
)

predictions_mlp_recursive

In [ ]:
predictions_combination = (predictions_mlp_multioutput + predictions_mlp_recursive) / 2

# ARIMA forecasts

In [ ]:
test.head()

In [ ]:
print(data[:n_learn])
print(data[n_learn:])

In [ ]:
model = auto_arima(data[:n_learn])
fcts_list = []
for i in np.arange(len(y_test)):
    pred = model.fit_predict(data[LAG + i : n_learn + LAG + i], n_periods=HORIZON)
    fcts_list.append(pred.to_numpy()[np.newaxis])

predictions_arima = pd.DataFrame(
    np.concatenate(fcts_list), columns=["t+" + str(t) for t in range(1, HORIZON + 1)]
)

# Forecast accuracy

In [ ]:
true_values = pd.DataFrame(
    test_inputs["target"], columns=["t+" + str(t) for t in range(1, HORIZON + 1)]
)

predictions = {
    'naive': predictions_naive,
    'mlp_multioutput': predictions_mlp_multioutput,
    'mlp_recursive': predictions_mlp_recursive,
    'combination': predictions_combination,
    'arima': predictions_arima,
}

results = {}
for model_name, prediction in predictions.items():
    results[model_name] = []
    for h in range(1, HORIZON + 1):
        time_horizon = "t+" + str(h)
        results[model_name].append(
            accuracy_measure(true_values[time_horizon], prediction[time_horizon])
        )

pd.DataFrame(results).mean().to_frame()